### (c) Igor Sokolov

In [84]:
%load_ext autoreload
%autoreload 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [58]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as sts
import math
import pandas as pd
import matplotlib
from matplotlib import patches
from sympy import *
import datetime
from functools import reduce
from sklearn.datasets import make_spd_matrix, make_sparse_spd_matrix
import scipy

from scipy.optimize import minimize, root_scalar

import sys
import os

import itertools
from scipy.special import binom
from scipy.stats import ortho_group

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import math
import sklearn.metrics 
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction import DictVectorizer as DV


from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score,roc_curve
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, KFold

from sklearn.model_selection import train_test_split
from sklearn.datasets import load_svmlight_file

from itertools import cycle
from contextlib import redirect_stdout
from functools import reduce

In [23]:
from logreg_functions import *

### Do a dataset preprocessing

In [82]:
%%bash
python3 generate_data.py --dataset mushrooms --loss_func log-reg --homogeneous 1

(8124,) [ 1.  1.  1. -1.  1.]
Data shape:  (8124, 112)


### Generate the script

In [90]:
project_path = os.getcwd() + "/"
experiment_name_ar = ["local_spider_homo"]
dataset_ar = ["mushrooms"]

batch_ar = [10]
LAUNCH_NUMBERS = 2
launch_number_ar = np.arange (1, LAUNCH_NUMBERS)
#num_workers_ar = [20, 50, 100]
num_workers_ar = [20]
#num_local_steps_ar = [10, 20, 30, 50, 100 ]
num_local_steps_ar = [10]
tol = 1e-16
release = True
max_it = 100
is_continue = 0


shell_script_name = "local_spider"

with open(shell_script_name + ".sh", 'w') as f:
    with redirect_stdout(f):
        #print ("%%bash")
        print ("#!/bin/bash")

        for launch_number in launch_number_ar:
            for i, (experiment_name, dataset, num_workers, num_local_steps, batch_size ) in \
            enumerate (itertools.product (experiment_name_ar, dataset_ar, num_workers_ar, num_local_steps_ar, batch_ar)):
                print ("python3 {0}.py --dataset {1} --num_workers {2} --num_local_steps {3} --batch_size {4} --continue {5} --max_it {6} --launch_number {7} --tol {8}".
                format(experiment_name, dataset, num_workers, num_local_steps, batch_size , is_continue, max_it, launch_number, tol))


                #show the script
f = open(shell_script_name + ".sh", 'r')
print (f.read())
f.close()



#!/bin/bash
python3 local_spider_homo.py --dataset mushrooms --num_workers 20 --num_local_steps 10 --batch_size 10 --continue 0 --max_it 100 --launch_number 1 --tol 1e-16



In [114]:
%%bash

bash local_spider.sh

2020-08-20 16:53:34
local_spider_homo_10_20_10


Traceback (most recent call last):
  File "local_spider_homo.py", line 272, in <module>
    V = sample_matrix_logreg_sgrad(W, X, y, la, i_batch) - sample_matrix_logreg_sgrad(W_prev, X, y, la, i_batch)  + V_prev # (7th)
  File "/Users/igorsokolov/Google_Drive/Sirius 2020/log_reg_experiment/logreg_functions.py", line 88, in sample_matrix_logreg_sgrad
    V[i] = sample_logreg_sgrad(W[i], X, y, la, i_batch)
  File "/Users/igorsokolov/Google_Drive/Sirius 2020/log_reg_experiment/logreg_functions.py", line 73, in sample_logreg_sgrad
    return np.sum(logreg_sgrad(w, X[i_batch], y[i_batch], la))/len(i_batch) + la * regularizer_grad(w)
  File "/Users/igorsokolov/Google_Drive/Sirius 2020/log_reg_experiment/logreg_functions.py", line 48, in logreg_sgrad
    loss_sgrad = - y_i * x_i / (1 + np.exp(y_i * np.dot(x_i, w)))
ValueError: operands could not be broadcast together with shapes (10,) (10,112) 


CalledProcessError: Command 'b'\nbash local_spider.sh\n'' returned non-zero exit status 1.